In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import pytorch_lightning as pl
import torch
import torch.nn as nn
from ay2.torch.deepfake_detection import DeepfakeAudioClassification
from ay2.torch.losses import BinaryTokenContrastLoss

In [27]:
from copy import deepcopy

In [5]:
from .model import AudioModel

In [5]:
from model import AudioModel

In [1]:
class AudioModel_lit(DeepfakeAudioClassification):
    def __init__(self, cfg=None, **kwargs):
        super().__init__()
        self.model = AudioModel()
        # ckpt = torch.load(
        #     "/home/ay/data/DATA/1-model_save/0-Audio/speech_emotion_recognition/version_1/model2.ckpt"
        # )
        # self.model.feature_model.load_state_dict(ckpt, strict=True)

        self.bce_loss = nn.BCEWithLogitsLoss()
        self.contrast_loss = BinaryTokenContrastLoss(alpha=0.4)
        self.cross_entropy_loss = nn.CrossEntropyLoss()
        self.save_hyperparameters()

    def calcuate_loss(self, batch_res, batch, stage="train"):
        B = batch_res["logit"].shape[0]

        label = batch["label"]
        label_32 = label.type(torch.float32)
        losses = {}
        losses["cls_loss"] = self.bce_loss(batch_res["logit"], label_32)
        # losses["emo_contrast_loss"] = self.contrast_loss(
        #     batch_res["emotion_feature"], label_32
        # )
        # losses["emo_cls_loss"] = self.bce_loss(batch_res["emotion_logit"], label_32)
        losses["vocoder_cls_loss"] = self.cross_entropy_loss(
            batch_res["vocoder_logit"], batch["vocoder_label"]
        )

        while losses["vocoder_cls_loss"] >= losses['cls_loss']:
            losses["vocoder_cls_loss"] = losses["vocoder_cls_loss"] / 2
        
        # losses["vocoder_contrast_loss"] = self.contrast_loss(
        #     batch_res["vocoder_feature"], label_32
        # )

        self.log_dict(
            {f"{stage}-{key}": losses[key] for key in losses},
            on_step=True,
            on_epoch=True,
            logger=True,
            prog_bar=True,
        )

        loss = [
            losses["cls_loss"],
            losses["vocoder_cls_loss"],
            # losses["vocoder_contrast_loss"],
        ]
        loss = torch.mean(torch.stack(loss))
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(
            self.model.parameters(), lr=0.00005, weight_decay=0.0001
        )
        return [optimizer]

    def _shared_pred(self, batch, batch_idx, stage="train"):
        audio, sample_rate = batch["audio"], batch["sample_rate"]
        batch_res = self.model(audio, stage=stage)
        batch_res["pred"] = (torch.sigmoid(batch_res["logit"]) + 0.5).int()
        return batch_res

    def _shared_eval_step(self, batch, batch_idx, stage="train"):
        batch_res = self._shared_pred(batch, batch_idx, stage=stage)

        label = batch["label"]
        loss = self.calcuate_loss(batch_res, batch, stage=stage)

        self.log_dict(
            {f"{stage}-loss": loss},
            on_step=False,
            on_epoch=True,
            logger=True,
            prog_bar=True,
        )
        batch_res["loss"] = loss
        # print(batch_res['pred'], batch_res['logit'], label)
        return batch_res

NameError: name 'DeepfakeAudioClassification' is not defined